In [3]:
import sys
sys.path.append('../../../src/')

import os
from pathlib import Path

from utils import parse, processing, plotting_utils as plotting, AddExtraColumns

# Plotting libraries
from matplotlib.backends.backend_pdf import PdfPages
import datetime
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

color1='#d95f02'
color2='#1b9e77'
color3='#7570b3'

odor_list_color = [color1, color2, color3]
color_dict = {1: color1, 0: color2, 2: color3}
color_dict_label = {'Ethyl Butyrate': color1, 'Alpha-pinene': color2, 'Amyl Acetate': color3}

pdf_path = r'Z:\scratch\vr-foraging\sessions'
base_path = 'Z:/scratch/vr-foraging/data/'
foraging_figures = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\VR foraging\results'

In [ ]:
date = datetime.date.today()
date_string = "05/01/2024"
date = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()

In [ ]:
mouse_list = ["672102", "694569", "690164","690165","690167","699894","699895","699899"]
mouse_list = ["715866", "713578", "707349", "716455", "716458","715865","715869","713545","715867","715870","716456","716457"]

In [ ]:
for mouse in mouse_list:
    print(mouse)
    session_found = False

    directory = os.path.join(base_path, mouse)
    files = os.listdir(os.path.join(base_path, mouse))
    
    sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)), reverse=True)
    
    for file_name in sorted_files:
        print(file_name)
        if session_found == True:
            break
        
        session_path = os.path.join(base_path, mouse, file_name)
        session = file_name[:8]
        session_path = Path(session_path)
        
        if datetime.date.fromtimestamp(os.path.getctime(session_path)) != date:
            continue
        else:
            print('correct date found')
            session_found = True
        
        data = parse.load_session_data(session_path)

        if 'tasklogic_input' in data['config'].streams.keys():
            tasklogic = 'tasklogic_input'
        else:
            tasklogic = 'TaskLogic'
            
        # Parse data
        reward_sites, active_site, config = parse.parse_dataframe(data)
        reward_sites = AddExtraColumns(reward_sites, active_site, run_on_init=True).reward_sites
        encoder_data = parse.ContinuousData(data).encoder_data

        last_engaged_patch = reward_sites['active_patch'][reward_sites['skipped_count'] >= 10].min()
        if pd.isna(last_engaged_patch):
            last_engaged_patch = reward_sites['active_patch'].max()
        reward_sites_cropped = reward_sites.loc[reward_sites['active_patch'] <= last_engaged_patch]
        
        trial_summary = plotting.trial_collection(reward_sites_cropped[['odor_sites', 'odor_label', 'has_choice', 'visit_number']], encoder_data, mouse, session, window=(-1,3))
        
        pdf_filename = mouse + '_' + session + '.pdf'

        # Save each figure to a separate page in the PDF
        with PdfPages(pdf_path+"\\"+pdf_filename) as pdf:
            try:
                plotting.length_distributions(active_site, data, delay=True, save=pdf)
            except:
                plotting.length_distributions(active_site, data, save=pdf)
                        
            plotting.speed_traces_efficient(trial_summary, mouse, session,  save=pdf)
            plotting.segmented_raster_vertical(reward_sites_cropped, data['config'].streams[tasklogic].data, save=pdf)
            
            try:
                plotting.speed_traces_available(trial_summary, mouse, session, data['config'], save=pdf)
                plotting.velocity_traces_odor_summary(trial_summary, data['config'], mouse, session, save=pdf)
                plotting.pstay_past_no_rewards(reward_sites_cropped, data['config'].streams['TaskLogic'].data, save=pdf)
                plotting.pstay_visit_number(reward_sites_cropped, data['config'].streams['TaskLogic'].data, save=pdf)
                
                # for x_start in range(0, int(sites.index[-1] - zero_index), 80):
                #     plotting.raster_plot(x_start, pdf)
                    
            except:
                pass
        
        # Summary of different relevants aspects -------------------------------------------------
        collected_df = reward_sites['collected'].sum()

        unrewarded_stops = reward_sites.loc[reward_sites.reward_delivered==0]['collected'].count()
        rewarded_stops = reward_sites.loc[reward_sites.reward_delivered==1]['collected'].count()
        water_collected = reward_sites.loc[(reward_sites['reward_delivered']==1)]['collected'].sum()
        total_stops = reward_sites.loc[(reward_sites['has_choice']==True)]['reward_available'].count()
        stopped_df = reward_sites.loc[(reward_sites['has_choice']==True)].groupby(['collected','odor_label'])[['reward_delivered']].sum().reset_index()

        # Rewarded stops / total available rewarded stops
        optimality = rewarded_stops / reward_sites[reward_sites.reward_available != 0]['reward_delivered'].count()

        # Rewarded stops / Stops
        efficiency = rewarded_stops / reward_sites.reward_delivered.sum()

        print('Total sites: ' ,len(reward_sites), ' | ', 'Total rewarded stops: ',rewarded_stops, '(',  np.round((rewarded_stops/total_stops)*100,2),'%) | ', 
            'Total unrewarded stops: ',unrewarded_stops,'(',  np.round((unrewarded_stops/total_stops)*100,2),'%) | ','Water consumed: ', water_collected, 'ul')

        if 'startPosition' in active_site.columns:
            stop_duration = np.round(active_site.startPosition.max()/100,2)
        else:
            stop_duration = np.round(active_site.start_position.max()/100,2)
        print('Total travelled m: ', np.round(active_site.start_position.max()/100,2))

        for odor_label in reward_sites.odor_label.unique():
            values = reward_sites.loc[(reward_sites['odor_label']==odor_label)&(reward_sites['reward_delivered']==1)]['collected'].sum()
            print(f'{odor_label} {values} ul')
            
    break
        

In [ ]:
plotting.segmented_raster_vertical(reward_sites_cropped, data['config'].streams[tasklogic].data)